<a href="https://colab.research.google.com/github/Trickshotblaster/nn-practices/blob/main/TrigramLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import math

In [27]:
import requests

url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
url = 'https://raw.githubusercontent.com/karpathy/makemore/master/names.txt'
response = requests.get(url)

with open('file.txt', 'wb') as f:
  f.write(response.content)

In [28]:
with open('file.txt', 'r') as data:
  dataset = data.readlines()

with open('file.txt', 'r') as data:
  raw_data = data.read()


In [29]:
raw_data[:10]

'emma\nolivi'

In [ ]:
dataset = raw_data.split("\n")

In [30]:
dataset[:10]

['emma\n',
 'olivia\n',
 'ava\n',
 'isabella\n',
 'sophia\n',
 'charlotte\n',
 'mia\n',
 'amelia\n',
 'harper\n',
 'evelyn\n']

In [31]:
vocab = set(raw_data)

In [32]:
vocab_len = len(vocab)

In [33]:
vocab_len

27

In [34]:
stoi = {char:i for i, char in enumerate(vocab)}

In [35]:
itos = {i:char for i, char in enumerate(vocab)}

In [36]:
import torch

In [37]:
import torch.nn.functional as functional

In [38]:
# make dataset
xs, ys = [], []
for data in dataset:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    xs.append([stoi[ch1], stoi[ch2]])
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
num

328158

In [39]:
ws = torch.randn((vocab_len*2, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([54, 27])

In [40]:
xenc = functional.one_hot(xs[0], num_classes=vocab_len).float().flatten()
xenc @ ws

tensor([-0.3484, -0.4287, -1.0911, -0.0913, -0.2678, -0.4413,  1.8398,  1.5626,
         1.4224, -1.9533,  0.9077, -0.9520, -0.2274, -0.5187,  0.8891, -0.0765,
        -0.5123, -0.4290, -0.4983,  0.4587,  0.5009,  1.0575, -0.2927,  0.5266,
         2.3705,  1.6488,  1.8706], grad_fn=<SqueezeBackward4>)

In [42]:
(xenc @ ws).shape

torch.Size([27])

In [43]:
num_epochs = 100
lr = 50

In [41]:
torch.randn((3, 1))

tensor([[-1.4262],
        [ 1.4987],
        [ 0.0610]])

In [44]:
for epoch in range(num_epochs):
    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * 0.01)
    print(loss.item())

    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr

4.002766132354736
3.356700897216797
3.089658260345459
2.923118829727173
2.813214063644409
2.7364096641540527
2.6803433895111084
2.6371471881866455
2.6025054454803467
2.5738236904144287
2.54961895942688
2.5288779735565186
2.51092267036438
2.4952244758605957
2.481394052505493
2.469111680984497
2.458134412765503
2.4482579231262207
2.4393253326416016
2.4312024116516113
2.423785448074341
2.4169833660125732
2.4107248783111572
2.4049463272094727
2.399597644805908
2.3946309089660645
2.3900108337402344
2.3857016563415527
2.3816757202148438
2.377906084060669
2.37437105178833
2.3710503578186035
2.3679254055023193
2.3649802207946777
2.3622004985809326
2.3595728874206543
2.357086181640625
2.354729413986206
2.352492570877075
2.350367546081543
2.348345994949341
2.3464207649230957
2.344585418701172
2.3428330421447754
2.3411595821380615
2.339559316635132
2.338027238845825
2.336560010910034
2.3351528644561768
2.3338027000427246
2.3325064182281494
2.3312599658966064
2.330061674118042
2.3289082050323486
2

In [25]:
xs[:10]

tensor([[ 4, 19],
        [19, 36],
        [36, 43],
        [43, 14],
        [14, 32],
        [32, 15],
        [15, 19],
        [19, 14],
        [14, 19],
        [19, 20]])

In [46]:
def forward(xs):
  xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
  logits = xenc @ ws
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  outchar = itos[torch.multinomial(probs, num_samples=1).item()]
  return outchar

def prompt(ins):
  out = ins
  for char in range(500):
    last_two = ins[-2:]
    xs = torch.tensor([[stoi[last_two[0]], stoi[last_two[1]]]])
    outs = forward(xs)
    out += outs
  return out

prompt("Rom")

'Roma\nue\neaaauya\na\n\nmroaabyioeamiea\n\naaeiaaaaioydei\n\niibfiaiha\niaeaiee\noyeay\nyoaeerubyauaaieaei\niaaea\n\nyoieii\nai\neai\nyaii\nuiaord\nakaiwre\n\neoyi\nai\n\n\ni\neaiapemaaiaai\nii\neanbaiea\neeeaiee\neiiaaa\nuaieimeuyeia\naysiiiuia\no\n\ni\nueeyi\ne\naaaa\n\noiu\nanapeaitaeeieyeezb\n\naimyeiiaejmaais\naiaaierieiioitoeaiiau\nyeyi\naangyiibia\ni\niiuysbiyyziaaywaoaoaaak\nes\n\nai\niipibaeqk\nraamaaiaoiiema\nieaieeli\n\nam\naa\ngaetbamaiaera\niaiaa\n\n\n\niraaeeiiea\naiaoaoiieami\nza\nyen\nyia\n\naeiaboipieaayui\niyaauaaeaoaaia\ni\n\nejei\neiaaaeoaaade\na\n\nazoai'